In [23]:
from face_lib.models.lightning_wrappers import ResNet
from face_lib.models.iresnet import IBasicBlock, conv1x1
from face_lib.models.heads import make_scf_layer
import torch

In [2]:
resnet = ResNet(
    resnet_name="iresnet50_normalized",
    weights="/app/model_weights/backbone/ms1mv3_arcface_r50/backbone.pth",
    learnable=False,
    use_cpu=True,
)

/app/face_lib/models/lightning_wrappers.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  backbone_dict = torch.load(weights, map_location=torch.device('cpu'))


In [10]:
x = torch.zeros(1, 3, 112, 112)
x = resnet.backbone.conv1(x)
x = resnet.backbone.bn1(x)
x = resnet.backbone.prelu(x)
layer1_out = resnet.backbone.layer1(x)
layer2_out = resnet.backbone.layer2(layer1_out)
layer3_out = resnet.backbone.layer3(layer2_out)

In [12]:
layer1_out.shape, layer2_out.shape, layer3_out.shape

(torch.Size([1, 64, 56, 56]),
 torch.Size([1, 128, 28, 28]),
 torch.Size([1, 256, 14, 14]))

In [37]:
layers = [4, 4, 4, 4]

In [38]:
from torch import nn

In [39]:
scf_layer1 = make_scf_layer(IBasicBlock, 64, 32, layers[0], stride=2)
scf_layer2 = make_scf_layer(IBasicBlock, 32, 32, layers[1], stride=2)
scf_layer3 = make_scf_layer(IBasicBlock, 32, 32, layers[2], stride=2)
scf_layer4 = make_scf_layer(IBasicBlock, 32, 32, layers[3], stride=2)
bn2 = nn.BatchNorm2d(
    512,
    eps=1e-05,
)
fc = nn.Linear(512, 1)
x = torch.flatten(x, 1)
bn3 = nn.BatchNorm1d(512, eps=1e-05)

scf_head = nn.Sequential(*[scf_layer1, scf_layer2, scf_layer3, scf_layer4])

In [40]:
from torchsummary import summary

In [42]:
# summary(scf_head, [64, 56, 56], device=torch.device('cpu'))

In [26]:
# scf_layer1_out = scf_layer1(layer1_out)
# scf_layer2_out = scf_layer2(scf_layer1_out)
# scf_layer3_out = scf_layer3(scf_layer2_out)

In [30]:
scf_head_out = scf_head(layer1_out)

In [31]:
scf_head_out.shape

torch.Size([1, 32, 4, 4])